In [26]:
!pip3 install jieba

    100% |████████████████████████████████| 7.3MB 1.5MB/s ta 0:00:011
  Running setup.py bdist_wheel for jieba ... done
  Stored in directory: /Users/nansu/Library/Caches/pip/wheels/c9/c7/63/a9ec0322ccc7c365fd51e475942a82395807186e94f0522243
Successfully built jieba


In [27]:
!python preprocess_translation/token_zh_en.py

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/q0/vtmm8ddj4vb30pptpc5ck6pc0000gn/T/jieba.cache
Loading model cost 0.750 seconds.
Prefix dict has been built succesfully.
dev.zh 100 1261
dev.zh 200 1261
dev.zh 300 1261
dev.zh 400 1261
dev.zh 500 1261
dev.zh 600 1261
dev.zh 700 1261
dev.zh 800 1261
dev.zh 900 1261
dev.zh 1000 1261
dev.zh 1100 1261
dev.zh 1200 1261
test.zh 100 1397
test.zh 200 1397
test.zh 300 1397
test.zh 400 1397
test.zh 500 1397
test.zh 600 1397
test.zh 700 1397
test.zh 800 1397
test.zh 900 1397
test.zh 1000 1397
test.zh 1100 1397
test.zh 1200 1397
test.zh 1300 1397
train.zh 100 213377
train.zh 200 213377
train.zh 300 213377
train.zh 400 213377
train.zh 500 213377
train.zh 600 213377
train.zh 700 213377
train.zh 800 213377
train.zh 900 213377
train.zh 1000 213377
train.zh 1100 213377
train.zh 1200 213377
train.zh 1300 213377
train.zh 1400 213377
train.zh 1500 213377
train.zh 1600 213377
train.zh 1700 213377
train.zh 1800 21

train.zh 35200 213377
train.zh 35300 213377
train.zh 35400 213377
train.zh 35500 213377
train.zh 35600 213377
train.zh 35700 213377
train.zh 35800 213377
train.zh 35900 213377
train.zh 36000 213377
train.zh 36100 213377
train.zh 36200 213377
train.zh 36300 213377
train.zh 36400 213377
train.zh 36500 213377
train.zh 36600 213377
train.zh 36700 213377
train.zh 36800 213377
train.zh 36900 213377
train.zh 37000 213377
train.zh 37100 213377
train.zh 37200 213377
train.zh 37300 213377
train.zh 37400 213377
train.zh 37500 213377
train.zh 37600 213377
train.zh 37700 213377
train.zh 37800 213377
train.zh 37900 213377
train.zh 38000 213377
train.zh 38100 213377
train.zh 38200 213377
train.zh 38300 213377
train.zh 38400 213377
train.zh 38500 213377
train.zh 38600 213377
train.zh 38700 213377
train.zh 38800 213377
train.zh 38900 213377
train.zh 39000 213377
train.zh 39100 213377
train.zh 39200 213377
train.zh 39300 213377
train.zh 39400 213377
train.zh 39500 213377
train.zh 39600 213377
train.zh 3

train.zh 72800 213377
train.zh 72900 213377
train.zh 73000 213377
train.zh 73100 213377
train.zh 73200 213377
train.zh 73300 213377
train.zh 73400 213377
train.zh 73500 213377
train.zh 73600 213377
train.zh 73700 213377
train.zh 73800 213377
train.zh 73900 213377
train.zh 74000 213377
train.zh 74100 213377
train.zh 74200 213377
train.zh 74300 213377
train.zh 74400 213377
train.zh 74500 213377
train.zh 74600 213377
train.zh 74700 213377
train.zh 74800 213377
train.zh 74900 213377
train.zh 75000 213377
train.zh 75100 213377
train.zh 75200 213377
train.zh 75300 213377
train.zh 75400 213377
train.zh 75500 213377
train.zh 75600 213377
train.zh 75700 213377
train.zh 75800 213377
train.zh 75900 213377
train.zh 76000 213377
train.zh 76100 213377
train.zh 76200 213377
train.zh 76300 213377
train.zh 76400 213377
train.zh 76500 213377
train.zh 76600 213377
train.zh 76700 213377
train.zh 76800 213377
train.zh 76900 213377
train.zh 77000 213377
train.zh 77100 213377
train.zh 77200 213377
train.zh 7

train.zh 109700 213377
train.zh 109800 213377
train.zh 109900 213377
train.zh 110000 213377
train.zh 110100 213377
train.zh 110200 213377
train.zh 110300 213377
train.zh 110400 213377
train.zh 110500 213377
train.zh 110600 213377
train.zh 110700 213377
train.zh 110800 213377
train.zh 110900 213377
train.zh 111000 213377
train.zh 111100 213377
train.zh 111200 213377
train.zh 111300 213377
train.zh 111400 213377
train.zh 111500 213377
train.zh 111600 213377
train.zh 111700 213377
train.zh 111800 213377
train.zh 111900 213377
train.zh 112000 213377
train.zh 112100 213377
train.zh 112200 213377
train.zh 112300 213377
train.zh 112400 213377
train.zh 112500 213377
train.zh 112600 213377
train.zh 112700 213377
train.zh 112800 213377
train.zh 112900 213377
train.zh 113000 213377
train.zh 113100 213377
train.zh 113200 213377
train.zh 113300 213377
train.zh 113400 213377
train.zh 113500 213377
train.zh 113600 213377
train.zh 113700 213377
train.zh 113800 213377
train.zh 113900 213377
train.zh 11

train.zh 145500 213377
train.zh 145600 213377
train.zh 145700 213377
train.zh 145800 213377
train.zh 145900 213377
train.zh 146000 213377
train.zh 146100 213377
train.zh 146200 213377
train.zh 146300 213377
train.zh 146400 213377
train.zh 146500 213377
train.zh 146600 213377
train.zh 146700 213377
train.zh 146800 213377
train.zh 146900 213377
train.zh 147000 213377
train.zh 147100 213377
train.zh 147200 213377
train.zh 147300 213377
train.zh 147400 213377
train.zh 147500 213377
train.zh 147600 213377
train.zh 147700 213377
train.zh 147800 213377
train.zh 147900 213377
train.zh 148000 213377
train.zh 148100 213377
train.zh 148200 213377
train.zh 148300 213377
train.zh 148400 213377
train.zh 148500 213377
train.zh 148600 213377
train.zh 148700 213377
train.zh 148800 213377
train.zh 148900 213377
train.zh 149000 213377
train.zh 149100 213377
train.zh 149200 213377
train.zh 149300 213377
train.zh 149400 213377
train.zh 149500 213377
train.zh 149600 213377
train.zh 149700 213377
train.zh 14

train.zh 181300 213377
train.zh 181400 213377
train.zh 181500 213377
train.zh 181600 213377
train.zh 181700 213377
train.zh 181800 213377
train.zh 181900 213377
train.zh 182000 213377
train.zh 182100 213377
train.zh 182200 213377
train.zh 182300 213377
train.zh 182400 213377
train.zh 182500 213377
train.zh 182600 213377
train.zh 182700 213377
train.zh 182800 213377
train.zh 182900 213377
train.zh 183000 213377
train.zh 183100 213377
train.zh 183200 213377
train.zh 183300 213377
train.zh 183400 213377
train.zh 183500 213377
train.zh 183600 213377
train.zh 183700 213377
train.zh 183800 213377
train.zh 183900 213377
train.zh 184000 213377
train.zh 184100 213377
train.zh 184200 213377
train.zh 184300 213377
train.zh 184400 213377
train.zh 184500 213377
train.zh 184600 213377
train.zh 184700 213377
train.zh 184800 213377
train.zh 184900 213377
train.zh 185000 213377
train.zh 185100 213377
train.zh 185200 213377
train.zh 185300 213377
train.zh 185400 213377
train.zh 185500 213377
train.zh 18

In [158]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import operator
from torch.utils.data import Dataset
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import sacrebleu

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PAD_token = 0
SOS_token = 1
EOS_token = 2
vocab_size = 85000
hidden_size = 256
# emb_size = 256
MAX_LENGTH = 100 # since 99% source sentence is <= 100
# MAX_LENGTH_1 = max(len(pair[0].split(" ")) for pair in pairs)
# MAX_LENGTH_2 = max(len(pair[1].split(" ")) for pair in pairs)
dropout_p = 0.1
teacher_forcing_ratio = 0.5
BATCH_SIZE = 32

## Loading Data

In [3]:
test = open('iwslt-zh-en/train.tok.zh').read().split('\n')

In [4]:
test[:10]

['深海 中 的 生命 － 大卫 . 盖罗 ',
 '大卫 . 盖罗 通过 潜水艇 拍下 的 影片 把 我们 带到 了 地球 最 黑暗 ， 最 险恶 同时 也 最 美丽 的 生物 栖息地 。 这里 是 海洋 深处 的 峡谷 和 火山 脊 ， 这里 怪诞 ， 适应力 强 而且 数量 惊人 的 生命 。 ',
 '大卫 . 盖罗 ： 这位 是 比尔 . 兰格 ，   我 是 大卫 . 盖罗 。 ',
 '我们 将 用 一些 影片 来 讲述 一些 深海 里 的 故事 。 ',
 '我们 这有 不少 精彩 的 泰坦尼克 的 影片 ，   可惜 您 今天 看不到 。 ',
 '泰坦尼克号   是 拿 了 不少 票房 冠军   但 事实上 它 并 不是 关于 海洋 的 最 刺激 的 故事 。 ',
 '原因 在于 我们 一直 没 把 海洋 当 回事儿 。 ',
 '大家 想想 ， 海洋 占 了 地球 面积 的 75 ％ 。 ',
 '地球 的 大部分 都 是 海水 。 ',
 '海洋 的 平均 深度 是 两英里 ']

In [5]:
test2 = open('iwslt-zh-en/train.tok.en').read().split('\n')

In [6]:
test2[:10]


['Life in the deep oceans',
 'With vibrant video clips captured by submarines , David Gallo takes us to some of Earth &apos;s darkest , most violent , toxic and beautiful habitats , the valleys and volcanic ridges of the oceans &apos; depths , where life is bizarre , resilient and shockingly abundant .',
 'This is Bill Lange . I &apos;m Dave Gallo .',
 'And we &apos;re going to tell you some stories from the sea here in video .',
 'We &apos;ve got some of the most incredible video of Titanic that &apos;s ever been seen , and we &apos;re not going to show you any of it .',
 'The truth of the matter is that the Titanic -- even though it &apos;s breaking all sorts of box office records -- it &apos;s not the most exciting story from the sea .',
 'And the problem , I think , is that we take the ocean for granted .',
 'When you think about it , the oceans are 75 percent of the planet .',
 'Most of the planet is ocean water .',
 'The average depth is about two miles .']

In [285]:
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS", 2:"EOS", 3:"UNK"}
        self.n_words = 4  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [349]:
def normalizeEnString(s):
#    s = unicodeToAscii(s.lower().strip())
#    s = re.sub(r"&apos;", r" ", s)
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s
def normalizeChString(s):
#    s = re.sub(r"([。！？])", r" \1", s)
#    s = re.sub(r"[.-*]+", r" ", s)
    return s.strip() 

In [350]:
normalizeEnString("It &apos;s very pretty , and it has rapidly started to overgrow the once very rich biodiversity of the northwestern Mediterranean .")

'It apos s very pretty and it has rapidly started to overgrow the once very rich biodiversity of the northwestern Mediterranean .'

In [351]:
def readLangs(lang1, lang2, data='train'):
    #data: train/dev/test
    print("Reading lines...")

    # Read the file and split into lines
    zh_lines = open('iwslt-zh-en/{}.tok.zh'.format(data)).read().split('\n')
    en_lines = open('iwslt-zh-en/{}.tok.en'.format(data)).read().split('\n')
    # Split every line into pairs and normalize
    pairs = [[normalizeChString(element[0]), normalizeEnString(element[1])] for element in zip(zh_lines, en_lines)]

    input_lang = Lang(lang1)
    output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [352]:
def prepareData(lang1, lang2):
    input_lang, output_lang, pairs = readLangs(lang1, lang2)
    print("Read %s sentence pairs" % len(pairs))
#    pairs = filterPairs(pairs)
#    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


In [353]:
def build_topwordVocab(lang, vocab_size):
    print("Build vocabulary by top {} frequent word...".format(vocab_size))
    sorted_word2Count = sorted(lang.word2count.items(),
        key=operator.itemgetter(1),
        reverse=True)
    sorted_words = [x[0] for x in sorted_word2Count[:vocab_size]]
    
#    print(sorted_words)
#    lang.index2word = {}
    lang.index2word[0] = "PAD"
    lang.index2word[1] = "SOS"
    lang.index2word[2] = "EOS"
    lang.index2word[3] = "UNK"
    
    for ind, word in enumerate(sorted_words):
            lang.word2index[word] = ind + 4
            

#    lang.word2index = {}
    for ind, word in enumerate(sorted_words):
        lang.index2word[ind + 4] = word
    
    lang.n_words = len(lang.index2word)
    
    print(lang.name, lang.n_words)
    return lang

input_lang, output_lang, pairs = prepareData('ch', 'eng')

input_lang = build_topwordVocab(input_lang,vocab_size)
print(random.choice(pairs))

Reading lines...
Read 213378 sentence pairs
Counting words...
Counted words:
ch 91144
eng 59373
Build vocabulary by top 85000 frequent word...
ch 91144
['所以 我 想要 这个 地球 上 的 每个 人 ，   都 能 成为 这个 设计 和 发展 的 一份 子 。', 'So I want everyone from all over the planet to be able to be a part of this design and development .']


In [361]:
_, _, val_pairs = readLangs('ch', 'eng', 'dev')

Reading lines...


In [363]:
val_pairs = val_pairs[:-1] # since last line is '',''

In [365]:
print(random.choice(val_pairs))

['首先 ， 为什么 他们 会 给 我们 留下 这种 坏 印象 呢 ？', 'First of all why do they have such a bad press ?']


In [366]:
val_input = [normalizeChString(line) for line in open('iwslt-zh-en/dev.tok.zh').read().split('\n')]
val_output = [normalizeEnString(line) for line in open('iwslt-zh-en/dev.tok.en').read().split('\n')]
val_inputs = []
val_outputs = []
for element in zip(val_input, val_output):
    val_inputs.append(element[0])
    val_outputs.append(element[1])

In [367]:
sorted_word2Count = sorted(input_lang.word2count.items(),
    key=operator.itemgetter(1),
    reverse=True)

In [368]:
sorted_word2Count ###标点符号排第一 之后要改掉

[('', 602339),
 ('的', 274644),
 ('，', 204840),
 ('。', 135957),
 ('我', 79407),
 ('我们', 68696),
 ('是', 66343),
 ('在', 59201),
 ('了', 53061),
 ('你', 40741),
 ('"', 34038),
 ('一个', 31867),
 ('这', 25708),
 ('有', 25464),
 ('他们', 25256),
 ('和', 25171),
 ('就', 19039),
 ('它', 18411),
 ('这个', 18294),
 ('都', 17439),
 ('他', 17395),
 ('可以', 17110),
 ('？', 16834),
 ('人', 16483),
 ('也', 15962),
 ('会', 15676),
 ('这些', 15481),
 ('不', 13342),
 ('上', 13266),
 ('-', 13070),
 ('中', 12900),
 ('到', 12803),
 ('说', 12625),
 ('：', 12143),
 ('做', 11929),
 ('就是', 11834),
 ('对', 11493),
 ('很', 11215),
 ('让', 10869),
 ('但', 10497),
 ('如果', 10347),
 ('因为', 9914),
 ('把', 9912),
 ('而', 9659),
 ('没有', 9504),
 ('来', 9440),
 ('能', 9361),
 ('—', 9199),
 ('所以', 9041),
 ('被', 8919),
 ('现在', 8805),
 ('想', 8447),
 ('什么', 8345),
 ('知道', 8338),
 ('这样', 8309),
 ('去', 8158),
 ('问题', 8062),
 ('但是', 8025),
 (',', 7831),
 ('从', 7823),
 ('她', 7792),
 ('当', 7721),
 ('这是', 7660),
 ('要', 7541),
 ('、', 7439),
 ('你们', 7391),
 ('那', 7365),

# Preparing Training Data

In [369]:
def indexesFromSentence(lang, sentence):
    idxs = []
    for word in sentence.split(' '):
        try:
            idxs.append(lang.word2index[word])
        except KeyError:
            idxs.append(3)  # 3 is the id of 'UNK'
    return idxs

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

class VocabDataset(Dataset):
    def __init__(self, pairs):
#         pairs = [tensorsFromPair(pair) for pair in pairs]
#         self.source_sent_list = [i[0] for i in pairs]
#         self.target_sent_list = [i[1] for i in pairs]
        
        self.source_sent_list = [indexesFromSentence(input_lang,pair[0]) for pair in pairs]
        self.target_sent_list = [indexesFromSentence(output_lang,pair[1]) for pair in pairs]
        
    def __len__(self):
        return len(self.source_sent_list)
        
    def __getitem__(self, key):
        token1_idx = self.source_sent_list[key][:MAX_LENGTH]
        token2_idx = self.target_sent_list[key][:MAX_LENGTH]
        return [token1_idx,token2_idx, len(token1_idx), len(token2_idx)]

    
def Vocab_collate_func(batch):
    source_sent_list = []
    target_sent_list = []
    source_len_list = []
    target_len_list = []

    for datum in batch:   ### batch = sample
        source_len_list.append(datum[2])
        target_len_list.append(datum[3])

    # padding
    for datum in batch:
        
        # source sentence processing
        padded_source = np.pad(np.array(datum[0]), 
                                pad_width=((0,MAX_LENGTH-datum[2])),          ### 0代表左边没有pad,右边的值代表右边pad的个数
                                mode="constant", constant_values=PAD_token)
        source_sent_list.append(padded_source)
        
        # target sentence processing
        padded_target = np.pad(np.array(datum[1]), 
                                pad_width=((0,MAX_LENGTH-datum[3])),          ### 0代表左边没有pad,右边的值代表右边pad的个数
                                mode="constant", constant_values=PAD_token)
        target_sent_list.append(padded_target)
        
    return [torch.tensor(source_sent_list,device = device), 
            torch.tensor(target_sent_list,device = device),
            torch.LongTensor(source_len_list,device = device), 
            torch.LongTensor(target_len_list,device = device)]

train_dataset = VocabDataset(pairs)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=Vocab_collate_func,
                                           shuffle=True)

val_dataset = VocabDataset(val_pairs)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                        batch_size=BATCH_SIZE,
                                        collate_fn=Vocab_collate_func,
                                        shuffle=False)

# Build Encoder-Decoder

In [400]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, bidirectional=True)
        self.linear = nn.Linear(2*hidden_size, hidden_size)
    def forward(self, input, hidden, batch_size):
        embedded = self.embedding(input).view(1, batch_size, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        output = self.linear(output)
        return output, hidden

    def initHidden(self, batch_size):
        return torch.zeros(2, batch_size, self.hidden_size, device=device)

In [411]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs, batch_size):
        embedded = self.embedding(input).view(1, batch_size, -1)
        embedded = self.dropout(embedded)
        
#         embedded: torch.Size([1, 32, 256])
#         hidden: torch.Size([1, 32, 256])
        
        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)   
#         attn_weights:torch.Size([32, 100])
        attn_applied = torch.bmm(attn_weights.unsqueeze(1),
                                 encoder_outputs.transpose(0,1))
#         encoder_outputs: 100*32*256 attn_applied: 32*1*256
        output = torch.cat((embedded[0], attn_applied.transpose(0,1)[0]), 1)

        # output: 32*768
        output = self.attn_combine(output).unsqueeze(0)
        # output 1*32*256
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = F.log_softmax(self.out(output[0]), dim=1)
#         output: torch.Size([32, 69126])
#         hidden: torch.Size([1, 32, 256])
        return output, hidden, attn_weights

# Training Model

In [43]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [443]:
# this is just one sentence input, could be batchlized 
def train(input_tensor, target_tensor, encoder, decoder,
          encoder_optimizer, decoder_optimizer, criterion, mask = None):
    batch_size = input_tensor.size(1)
    encoder_hidden = encoder.initHidden(batch_size)
    encoder_optimizer.zero_grad()  # zero out the accumulated gradient over mini-batch
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0) # length of source sentence
    target_length = target_tensor.size(0)
    encoder_outputs = torch.zeros(target_length, batch_size, encoder.hidden_size, device=device) 
    # (seq_length, BATCH_SIZE,hidden_size*2) 2 due to bidirection

    loss = 0
    
#    print(batch_size)
    # feed-forward layer resulting encoder outputs, ei refers to each word token in input sentence
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden, batch_size)  
        # encoder_output: torch.Size([1, 32, 512]) encoder_hidden: torch.Size([2, 32, 256])
        encoder_outputs[ei] = encoder_output[0] 
    # change the shape of encoder output to fit into decoder 
    encoder_hidden = nn.Linear(2*hidden_size,hidden_size)(
        torch.cat((encoder_hidden[0],encoder_hidden[1]),dim = 1)).unsqueeze(0)
    
    decoder_input = torch.tensor([[SOS_token]*batch_size], device=device)  # decoder_input: torch.Size([1, 32])
    # init decoder hidden 
    decoder_hidden = encoder_hidden
    
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs, batch_size)
            
            decoder_input = target_tensor[di]  # Teacher forcing
            temp_loss = criterion(decoder_output, target_tensor[di])
            loss += temp_loss * mask[di:di+1].float()  
            ave_loss = loss.sum()/batch_size 
            
    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            # decoder_input: torch.Size([1, 32])
            # decoder_hidden: torch.Size([1, 32, 256]) 1 token * batch * hidden size
            # encoder_outputs: torch.Size([100, 32, 512])
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs, batch_size)
            topv, topi = decoder_output.topk(1)
            # topv: 32*1
            # topi: 32*1

            decoder_input = topi.squeeze().detach()  # detach from history as input
            # decoder_input: 32
            # target_tensor: 100*32
            # decoder_output: 32*69127 
            temp_loss = criterion(decoder_output, target_tensor[di])
            loss += temp_loss * mask[di:di+1].float()
            # loss size 1*32
            ave_loss = loss.sum()/batch_size  
            
    ave_loss.backward()
    
    
    encoder_optimizer.step()   # update parameters
    decoder_optimizer.step()

    return ave_loss.item() / target_length

In [464]:
def trainIters(encoder, decoder, n_iters, print_every=100, plot_every=100, learning_rate=0.001):
    start = time.time()

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss(reduce = False) ##!!!!!!!!!!1 这个loss是否要换成crossentropy

    for epoch in range(1, n_iters + 1):
        plot_losses = []
        print_loss_total = 0  # Reset every print_every
        plot_loss_total = 0  # Reset every plot_every
        for i, (input_sentences, target_sentences,len1,len2) in enumerate(train_loader): 
            input_tensor = input_sentences.transpose(0,1)   # 13*100 to 100*13
            target_tensor = target_sentences.transpose(0,1)
            mask = target_tensor.ge(1)   # 100 * 13
            loss = train(input_tensor, target_tensor, encoder,
                         decoder, encoder_optimizer, decoder_optimizer, criterion, mask = mask)
            print_loss_total += loss
            plot_loss_total += loss

            if i > 0 and i % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                bleu_score, (sys_sents, ref_sents) = test_model(encoder, decoder, val_loader)
                print('Time: {}, Epoch: [{}/{}], Step: [{}/{}], Train Loss: {}, Validation Score: {} \n Predicted sentence: {} \n Reference sentence: {}'.format(
                    timeSince(start, i + 1/len(train_loader)), epoch, n_iters, i, 
                    len(train_loader),print_loss_avg, bleu_score, sys_sents, ref_sents))

            if i > 0 and i % plot_every == 0:
                plot_loss_avg = plot_loss_total / plot_every
                plot_losses.append(plot_loss_avg)
                plot_loss_total = 0
                
        print(plot_losses)
        showPlot(plot_losses)

# Plotting results

In [441]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

# Evaluation

In [436]:
def get_batch_outputs(encoder, decoder, input_sentences, max_length=MAX_LENGTH): 
    with torch.no_grad():
        input_tensor = input_sentences.transpose(0,1)   # 32*100 to 100*32
        batch_size = input_tensor.size(1)
        encoder_hidden = encoder.initHidden(batch_size)

        input_length = input_tensor.size(0) # length of source sentence
        encoder_outputs = torch.zeros(max_length, batch_size, encoder.hidden_size, device=device) 
        
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], 
                                                     encoder_hidden, batch_size)  
            encoder_outputs[ei] = encoder_output[0] 
            
        encoder_hidden = nn.Linear(2*hidden_size,hidden_size)(
            torch.cat((encoder_hidden[0],encoder_hidden[1]),dim = 1)).unsqueeze(0)

        decoder_input = torch.tensor([[SOS_token]*batch_size], device=device)  # decoder_input: torch.Size([1, 32])
        decoder_hidden = encoder_hidden
        
        decoded_words = np.empty((max_length, batch_size), dtype=object)
        
        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs, batch_size)
            topv, topi = decoder_output.topk(1)

#            topi_lst = topi.squeeze().detach().tolist()
            decoder_input = topi.squeeze().detach()  # detach from history as input
            
            decoded_words[di:] = np.array(['<EOS>' if idx==EOS_token else output_lang.index2word[idx] for idx in decoder_input.tolist()])
        
        return decoded_words.transpose()
        

In [461]:
def test_model(encoder, decoder, loader):
    score = []
    for i, (input_sentences, target_sentences, len1, len2) in enumerate(loader):
        sys_sentences = [' '.join(sentence) for sentence in get_batch_outputs(encoder, decoder, input_sentences)]
        ref_sentences = [val_pair[1] for val_pair in val_pairs[i*32:(i+1)*32]]
        score_batch = [sacrebleu.corpus_bleu([sys],[[ref]]).score for sys, ref in zip(sys_sentences, ref_sentences)]
        score.append(sum(score_batch)/len(score_batch))
    return sum(score)/len(score), (sys_sentences[0], ref_sentences[0])


In [162]:
# def evaluateRandomly(encoder, decoder, n=10):
#     for i in range(n):
#         pair = random.choice(pairs)
#         print('>', pair[0])
#         print('=', pair[1])
#         output_words = generate_output(encoder, decoder, pairs)
#         output_sentence = ' '.join(output_words)
#         print('<', output_sentence)
#         print('')

# TRAINING AND EVALUATING

In [ ]:
encoder1 = EncoderRNN(input_lang.n_words,hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.5).to(device)

trainIters(encoder1, attn_decoder1, 3, print_every=10,plot_every=1)

torch.save(encoder1.state_dict(), "saved_model/encoder_hiddenSize{}".format(hidden_size))
torch.save(attn_decoder1.state_dict(), "saved_model/attn_decoder_hiddenSize{}".format(hidden_size))

/Users/nansu/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


Time: 4m 24s (- -4m 2s), Epoch: [1/3], Step: [10/6669], Train Loss: 2.0456244659423826, Validation Score: 0.23895635721809186 
 Predicted sentence: of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of of 
 Reference sentence: They apos re also ways for us to freeze time for one week in October and reflect on our times and how we change from year to year and not just physically but in every way .
Time: 8m 31s (- -9m 54s), Epoch: [1/3], Step: [20/6669], Train Loss: 1.4231763458251954, Validation Score: 0.35523767936343437 
 Predicted sentence: . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
 Refere